In [1]:
# It is important to check which env you are working with 
import sys
print(sys.executable)

/home/fatnaoui/tools/miniforge3/envs/argilla/bin/python


In [ ]:
# Install dependencies 
! pip install -r requirements.txt

In [2]:
# Check your installation 
!pip show argilla

Name: argilla
Version: 2.8.0
Summary: The Argilla python server SDK
Home-page: 
Author: 
Author-email: Argilla <contact@argilla.io>
License: Apache 2.0
Location: /home/fatnaoui/tools/miniforge3/envs/argilla/lib/python3.14/site-packages
Requires: datasets, httpx, huggingface_hub, pillow, pydantic, rich, standardwebhooks, tqdm
Required-by: 


In [2]:
API_URL = "http://localhost:6900"
API_KEY = "fatnaoui.apikey"

import argilla as rg

client = rg.Argilla(
    api_url=API_URL,
    api_key=API_KEY,
)

me = client.me
me

/home/fatnaoui/tools/miniforge3/envs/argilla/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


User(id=UUID('e20402cb-606e-4e73-b913-8ad65e5c64d4') inserted_at=datetime.datetime(2025, 12, 13, 14, 23, 24, 69851) updated_at=datetime.datetime(2025, 12, 13, 14, 23, 24, 69851) username='fatnaoui' role=<Role.owner: 'owner'> first_name='fatnaoui' last_name=None password=None)

In [12]:
# List existing workspaces
workspaces = client.workspaces.list()
for ws in workspaces:
    print(ws.name)

In [49]:
# workspace_to_delete = client.workspaces("langID_validation")

# deleted_workspace = workspace_to_delete.delete()

workspace_to_delete = client.workspaces("langID_Annotation")

deleted_workspace = workspace_to_delete.delete()

In [50]:
# Create a Workspace
workspace_to_create = rg.Workspace(name="langID_Annotation")

created_workspace = workspace_to_create.create()

workspace_to_create = rg.Workspace(name="langID_validation")

created_workspace = workspace_to_create.create()

workspaces = client.workspaces.list()
for ws in workspaces:
    print(ws.name)

langID_Annotation
langID_validation


In [51]:
# Dataset settings 
settings1 = rg.Settings(
    guidelines="""
You are labeling text examples to identify the language.
Choose the correct language among:
- Darija (Moroccan Arabic)
- MSA (Modern Standard Arabic)
- French
- English

If the text contains mixed languages, choose the *dominant* one.
""",
    fields=[
        rg.TextField(
            name="text",
            title="Input Text", # added
            description="This is the text you must classify by language.", # added
            required=True,
        ),
    ],
    questions=[
        rg.LabelQuestion(
            name="language",
            title="Which language is used here?",
            description="Choose the language that best fits the text.",
            labels=["darija", "msa", "french", "english"],
            required=True,
        ),
    ],
    metadata=[
            rg.TermsMetadataProperty(name="category", options=["origin", "validated"]),
        ],
)

settings2 = rg.Settings(
    guidelines="""
You are validating 
""",
    fields=[
        rg.TextField(
            name="text",
            title="Input text", # added
            description="This is the text you must classify by language.", # added
            required=True,
        ),
        rg.TextField(
            name="language",
            title="Input language", # added
            required=True,
        )
    ],
    questions=[
        rg.LabelQuestion(
            name="Validation",
            title="Is this valid or not ?",
            labels=["Valid", "Not Valid"],
            required=True,
        ),
    ],
    metadata=[
            rg.TermsMetadataProperty(name="category", options=["origin", "validated"]),
        ],
)

# Create dataset
dataset1 = rg.Dataset(
    name="Darija_Identification",
    workspace="langID_Annotation",
    settings=settings1,
)

dataset2 = rg.Dataset(
    name="Darija_validation",
    workspace="langID_validation",
    settings=settings2,
)

dataset1.create()
dataset2.create()

Dataset(id=UUID('caef6b9e-0692-4873-a981-482cfd7005d5') inserted_at=datetime.datetime(2025, 12, 13, 15, 20, 17, 407971) updated_at=datetime.datetime(2025, 12, 13, 15, 20, 17, 778127) name='Darija_validation' status='ready' guidelines='You are validating' allow_extra_metadata=False distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('8027caf5-90a0-46e4-92fe-b01064ad8abf') last_activity_at=datetime.datetime(2025, 12, 13, 15, 20, 17, 778127))

In [40]:
# Delete Dataset
# dataset_to_delete = client.datasets(name="Darija_Identification")

# dataset_deleted = dataset_to_delete.delete()

dataset_to_delete = client.datasets(name="Darija_validation", workspace="langID_validation")

dataset_deleted = dataset_to_delete.delete()

In [25]:
# List all users 
users = client.users

for user in users:
    print(user)
    
# List users in a workspace
workspace = client.workspaces('langID_validation')

for user in workspace.users:
    print(user)

User(id=UUID('e20402cb-606e-4e73-b913-8ad65e5c64d4') inserted_at=datetime.datetime(2025, 12, 13, 14, 23, 24, 69851) updated_at=datetime.datetime(2025, 12, 13, 14, 23, 24, 69851) username='fatnaoui' role=<Role.owner: 'owner'> first_name='fatnaoui' last_name=None password=None)
User(id=UUID('b052db3c-484f-4773-b53f-5062d418f471') inserted_at=datetime.datetime(2025, 12, 13, 14, 35, 28, 695553) updated_at=datetime.datetime(2025, 12, 13, 14, 35, 28, 695553) username='hamza' role=<Role.annotator: 'annotator'> first_name='hamza' last_name=None password=None)


In [54]:
# It will take role annotator automatically 
user_to_create = rg.User(
    username="hamza",
    password="hamza123",
)

created_user = user_to_create.create()

In [53]:
user_to_delete = client.users('hamza')

deleted_user = user_to_delete.delete()

In [56]:
# Add user to a workspace
user = client.users('hamza')
workspace = client.workspaces('langID_Annotation')

added_user = user.add_to_workspace(workspace)

In [57]:
# Add record using a generic data structure 
dataset = client.datasets(name="Darija_Identification")

# Add records to the dataset with the fields 'question' and 'answer'
data = [
    {
        "text": "تقدر تعاوني؟ تقدري تعاونيني؟",
        "category": "origin"
    },
    {
        "text": "this is hamza hello",
        "category": "origin"
    }, 
]
dataset.records.log(data)

DatasetRecords: The provided batch size 256 was normalized. Using value 2.

Sending records...: 100%|██████████| 1/1 [00:00<00:00,  7.13batch/s]


DatasetRecords(Dataset(id=UUID('3ec30367-2b67-4481-bf5c-002120dd85da') inserted_at=datetime.datetime(2025, 12, 13, 15, 20, 16, 486877) updated_at=datetime.datetime(2025, 12, 13, 15, 20, 16, 705761) name='Darija_Identification' status='ready' guidelines='You are labeling text examples to identify the language.\nChoose the correct language among:\n- Darija (Moroccan Arabic)\n- MSA (Modern Standard Arabic)\n- French\n- English\n\nIf the text contains mixed languages, choose the *dominant* one.' allow_extra_metadata=False distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('c5d38b65-6f6d-43a5-ad16-c5cac5f0fec8') last_activity_at=datetime.datetime(2025, 12, 13, 15, 20, 16, 705761)))

In [58]:
# Track team progress 
import argilla as rg

dataset = client.datasets("Darija_Identification")

progress = dataset.progress(with_users_distribution=True)

progress

{'total': 2, 'completed': 0, 'pending': 2, 'users': {}}

In [ ]:
# Import your dataset to your disk
import argilla as rg

dataset = client.datasets(name="Darija_Identification")

dataset.to_disk(path="darija", with_records=False)

In [ ]:
# Export records as a dictionary with flatten=True
dataset = client.datasets(name="Darija_Identification")

exported_records = dataset.records.to_dict(flatten=True)

exported_records

In [10]:
keys_to_include = ['text', 'language.responses']

records = {key: exported_records[key] for key in keys_to_include if key in exported_records}

records

NameError: name 'exported_records' is not defined

In [ ]:
# Save your data
import json

file_path = "identification_darija_data.json"
with open(file_path, "w") as f:
    json.dump(records, f, indent=4, ensure_ascii=False)

In [59]:
for webhook in client.webhooks:
    print(webhook)

In [36]:
for webhook in client.webhooks:
    webhook.delete()

In [129]:
import argilla as rg

client = rg.Argilla(api_url="http://localhost:6900", api_key="fatnaoui.apikey")

print("Argilla version:", rg.__version__)
for wh in client.webhooks:
    print("URL:", wh.url)
    print("EVENTS:", wh.events)
    print("ENABLED:", wh.enabled)
    print("-" * 30)


Argilla version: 2.8.0
URL: http://host.docker.internal:8000/my_webhook_handler
EVENTS: [<EventType.dataset_created: 'dataset.created'>]
ENABLED: True
------------------------------
